In [19]:
# !nvidia-smi
!nvidia-smi --query-gpu=name --format=csv,noheader | wc -l

/usr/bin/sh: nvidia-smi: command not found
0


In [20]:
import os
import pandas as pd
import tqdm
import math
import torch
from torch.utils.data.dataset import Dataset

In [21]:
#Set the path to the data folder, datafile and output folder and files

op_folder_name = 'sept2022'

root_folder = '/users/kent/jmaharja/drugAbuse/'
output_folder = os.path.abspath(os.path.join(root_folder, 'output/'+ op_folder_name))
model_folder = os.path.abspath(os.path.join(output_folder, 'RoBERTaMLM/'))
tokenizer_folder = os.path.abspath(os.path.join(output_folder, 'TokRoBERTa/'))

datafile= '2020_01_01.csv'
testfile= '20161007.csv'
outputfile = 'submission.csv'

input_folder = os.path.abspath(os.path.join(root_folder, 'input/'))
datafile_path = os.path.abspath(os.path.join(input_folder, datafile))
testfile_path = os.path.abspath(os.path.join(input_folder, testfile))
outputfile_path = os.path.abspath(os.path.join(output_folder, outputfile))

In [22]:
train_df =pd.read_csv(datafile_path,lineterminator='\n',skipinitialspace=True, usecols= ['text'])
train_df.rename(columns={'text':'Tweet'}, inplace=True)
train_df = train_df.dropna()
train_df.shape

(1115630, 1)

# Train a language model from scratch

In [23]:
TRAIN_BATCH_SIZE = 16    # input batch size for training (default: 64)
VALID_BATCH_SIZE = 8    # input batch size for testing (default: 1000)
TRAIN_EPOCHS = 1        # number of epochs to train (default: 10)
LEARNING_RATE = 1e-4    # learning rate (default: 0.001)
WEIGHT_DECAY = 0.01
SEED = 42               # random seed (default: 42)
MAX_LEN = 128
SUMMARY_LEN = 7

In [24]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=8192,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [25]:
from transformers import RobertaForMaskedLM
model = RobertaForMaskedLM(config=config)
print('Num parameters: ', model.num_parameters())

Num parameters:  49816064


In [26]:
# Check that PyTorch sees it
if torch.cuda.is_available():
    print("Let's use", torch.cuda.device_count(), "GPUs!")
#     model = torch.nn.DataParallel(model)


In [27]:
from transformers import RobertaTokenizerFast
# Create the tokenizer from a trained one
tokenizer = RobertaTokenizerFast.from_pretrained(tokenizer_folder, max_len=MAX_LEN)

Didn't find file /users/kent/jmaharja/drugAbuse/output/sept2022/TokRoBERTa/tokenizer.json. We won't load it.
Didn't find file /users/kent/jmaharja/drugAbuse/output/sept2022/TokRoBERTa/added_tokens.json. We won't load it.
Didn't find file /users/kent/jmaharja/drugAbuse/output/sept2022/TokRoBERTa/special_tokens_map.json. We won't load it.
Didn't find file /users/kent/jmaharja/drugAbuse/output/sept2022/TokRoBERTa/tokenizer_config.json. We won't load it.
loading file /users/kent/jmaharja/drugAbuse/output/sept2022/TokRoBERTa/vocab.json
loading file /users/kent/jmaharja/drugAbuse/output/sept2022/TokRoBERTa/merges.txt
loading file None
loading file None
loading file None
loading file None


In [28]:
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42
train_df, test_df = train_test_split(train_df, test_size=0.1, random_state=RANDOM_SEED)
val_df, test_df = train_test_split(test_df, test_size=0.5, random_state=RANDOM_SEED)

# Building the training Dataset

In [29]:
class CustomDataset(Dataset):
    def __init__(self, df, tokenizer):
        # or use the RobertaTokenizer from `transformers` directly.

        self.examples = []
        
        for example in df.values:
            x=tokenizer.encode_plus(example, max_length = MAX_LEN, truncation=True, padding=True)
            self.examples += [x.input_ids]

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        # We’ll pad at the batch level.
        return torch.tensor(self.examples[i])

In [30]:
# Create the train and evaluation dataset
train_dataset = CustomDataset(train_df['Tweet'], tokenizer)
eval_dataset = CustomDataset(val_df['Tweet'], tokenizer)

In [31]:
train_df.iloc[0]
train_dataset.__getitem__(0)

tensor([   0,   54,   56,  265,  979,  675,  543, 3326,   94,   30,  933,  402,
        1447,  396, 5212,   18,  933,  402, 1447, 1563,   18,  933,  402, 1447,
        7672,  619,  358, 7074,   18,  933,  402, 1447,  795, 1503, 2240,   18,
         933,  402,  307,    2])

In [32]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

In [33]:
from torch import nn
from transformers import Trainer, TrainingArguments

In [34]:
#from transformers import Trainer, TrainingArguments
print(model_folder)
# Define the training arguments
training_args = TrainingArguments(
    output_dir=model_folder,
    overwrite_output_dir=True,
    evaluation_strategy = 'epoch',
    num_train_epochs=TRAIN_EPOCHS,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=VALID_BATCH_SIZE,
    save_steps=8192,
    #eval_steps=4096,
    save_total_limit=1,
)
# Create the trainer for our model
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    #prediction_loss_only=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


/users/kent/jmaharja/drugAbuse/output/sept2022/RoBERTaMLM


In [ ]:
# Train the model
trainer.train()

/users/kent/jmaharja/.local/lib/python3.6/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1004067
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 62755


Epoch,Training Loss,Validation Loss


In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
trainer.save_model(model_folder)